In [5]:
data_dict = {}
with open("data/110_oily_portraits.txt", "r") as file:
    for i, line in enumerate(file):
        data_dict[i] = line.strip().split()
        data = file.readlines()
print(data_dict)

{0: ['80000']}


In [6]:
# Extract a subset of data_dict
subset_data_dict = {k: v for k, v in data_dict.items() if k <= 10}
print(subset_data_dict)

{0: ['80000']}


In [7]:
transformed_data_dict = {}

def transform_data(row):
    parts = row
    return {
        "type": parts[0] if len(parts) > 0 else None,
        "tagNumber": int(parts[1]) if len(parts) > 1 else None,
        "tags": parts[2:] if len(parts) > 2 else []
    }

for key, value in data_dict.items():
    transformed_data_dict[key] = transform_data(value)
    
    if key > 10:
        break

print(transformed_data_dict)

{0: {'type': '80000', 'tagNumber': None, 'tags': []}}


In [2]:
%pip install numpy
import numpy as np

# Function to calculate the similarity between all pairs
def compute_similarity_matrix(entries):
    n = len(entries)
    similarity_matrix = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(i + 1, n):  # Fill the upper triangle
            similarity = len(set(entries[i].split()) & set(entries[j].split()))
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity  # Mirror the value
    return similarity_matrix

# Pair entries using the similarity matrix
def pair_entries_with_matrix(entries):
    similarity_matrix = compute_similarity_matrix(entries)
    pairs = []
    used_indices = set()
    n = len(entries)

    for _ in range(n // 2):  # We can pair at most n // 2 times
        max_sim = -1
        best_pair = None

        # Find the best pair (most similar)
        for i in range(n):
            if i in used_indices:
                continue
            for j in range(i + 1, n):
                if j in used_indices:
                    continue
                if similarity_matrix[i, j] > max_sim:
                    max_sim = similarity_matrix[i, j]
                    best_pair = (i, j)
        
        if best_pair:
            i, j = best_pair
            pairs.append((entries[i], entries[j]))
            used_indices.update({i, j})
        else:
            break  # No more pairs can be formed

    # Handle leftover entries
    leftover = [entries[i] for i in range(n) if i not in used_indices]

    return pairs, leftover

# Extract P entries and their contents
p_entries = [line.split(maxsplit=2)[2].strip() for line in data if line.startswith("P")]

# Execute the optimized pairing
pairs, leftover = pair_entries_with_matrix(p_entries)

# Output the results
print("Grouped P Entries:")
for idx, (entry1, entry2) in enumerate(pairs, 1):
    print(f"Pair {idx}:")
    print(f"  {entry1}")
    print(f"  {entry2}")

if leftover:
    print("\nLeftover P Entries:")
    for entry in leftover:
        print(f"  {entry}")


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


NameError: name 'data' is not defined